# First load os and sys so I can update the sys.path with new functions

change the luminosity to 80 /fb

generate the 3 plots as in aewol paper.

understant the error in asimov function.

In [ ]:
import os
import sys

In [ ]:
#take the paths to the functions we nedd
module_path = os.path.abspath(os.path.join('./pandasPlotting/'))
module2_path = os.path.abspath(os.path.join('./MlClasses/'))
module3_path = os.path.abspath(os.path.join('./MlFunctions/'))

In [ ]:
# this part will include in the sys.path variables the paths for our new functions
if [module_path, module2_path, module3_path] not in sys.path:
    sys.path.append(module_path)

In [ ]:
# here we are going to load what we will need, keras + tensorflow, plot functions, etc..
import matplotlib
matplotlib.use('Agg')
import pandas as pd
import numpy as np
import math
import time

from keras import callbacks

from pandasPlotting.Plotter import Plotter
from pandasPlotting.dfFunctions import expandArrays
from pandasPlotting.dtFunctions import featureImportance

from MlClasses.MlData import MlData
from MlClasses.Bdt import Bdt
from MlClasses.Dnn import Dnn
from MlClasses.ComparePerformances import ComparePerformances

from MlFunctions.DnnFunctions import significanceLoss,significanceLossInvert,significanceLoss2Invert ,significanceLossInvertSqrt,significanceFull,asimovSignificanceLoss,asimovSignificanceLossInvert,asimovSignificanceFull,truePositive,falsePositive

from linearAlgebraFunctions import gram,addGramToFlatDF

In [ ]:
# I don't have patience to training 200 epochs ¯\_(ツ)_/¯
earlyStopping = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5)

In [ ]:
#load our data files
signal=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/0L/feb/feb-0L-inclusive/zh0p030lfeb.csv",sep='\s+',engine='python')
bkgd=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/0L/feb/feb-0L-inclusive/zhsm0lfeb.csv",sep='\s+',engine='python')


In [ ]:
#combine them into one dataset
combined = pd.concat([signal,bkgd]).sample(frac=1)

In [ ]:
print(combined.keys())

In [ ]:
# change thes vars depend on which dataset you are loading, I will implement a better solution.
chosenVars = {
            '0L':['ptb1', 'ptb2', 'misset', 'pth', 'ptz', 'etah', 'phih', 'mtvh', 'ptvh', 'dphib1met', 'signal'],
            #'2L':['ptb1', 'ptb2', 'ptl1', 'ptl2', 'pth', 'ptz', 'etah', 'phih',
            #     'deltarll', 'deltarbl', 'mtvh', 'ptvh', 'dphil1b1', 'dphil1b2', 'signal']
}

In [ ]:
trainedModels={}

In [ ]:
#needed to plot asimov significane
asimovSigLossSysts=[0.01,0.05,0.1,0.2,0.3,0.4,0.5]

In [ ]:
# here I have included one archtecture I got from my ES scan, pls comment my entry and use dnn_batch4096 instead.
dnnConfigs={
    #'dnn_ZH_0L_cHW_0d001_batch_1024':{'epochs':200,'batch_size':4096,'dropOut':None,
    #                                  'l2Regularization':None,'hiddenLayers':[1.0],
    #                                  'optimizer':'adam', 'activation':'relu'}
    #'dnn_ZH_0L_cHW_0d01_batch_1024':{'epochs':200,'batch_size':4096,'dropOut':None,
    #                                  'l2Regularization':None,'hiddenLayers':[1.0],
    #                                  'optimizer':'adam', 'activation':'relu'}
    'dnn_ZH_0L_cHW_0d03_batch_1024':{'epochs':200,'batch_size':4096,'dropOut':None,
                                      'l2Regularization':None,'hiddenLayers':[1.0],
                                      'optimizer':'adam', 'activation':'relu'}
        }

In [ ]:
#bsmlike signal cHW 0.03: 11.867fb, bg:0.89
#inclusive signal cHW 0.001: 2.198fb, bg:2.03fb
#inclusive signal cHW 0.01: 4.553fb, bg:2.03fb
#inclusive signal cHW 0.03: 14fb, bg:2.03fb

lumi=80. #luminosity in /fb
expectedSignal= 14*lumi 
expectedBkgd=2.03*lumi #cross section of ttbar sample in fb times efficiency measured by Marco
systematic=0.5 #systematic for the asimov signficance

In [ ]:
# I'm only running the DNN with binary cross entropy, letter I will include the other options.
# so far it running perfect, I'm testing with their custom loss function.
for varSetName,varSet in chosenVars.items():
    #Pick out the expanded arrays
    columnsInDataFrame = []
    for k in combined.keys():
        for v in varSet:
            #Little trick to ensure only the start of the string is checked
            if varSetName is '0L':
                if ' '+v+' ' in ' '+k+' ': columnsInDataFrame.append(k)
            elif ' '+v in ' '+k: columnsInDataFrame.append(k)


    #Select just the features we're interested in
    #For now setting NaNs to 0 for compatibility
    combinedToRun = combined[columnsInDataFrame].copy()
    combinedToRun.fillna(0,inplace=True)
    
    combinedToRun.index = np.arange(0,200000)
    mlData = MlData(combinedToRun,'signal')

    mlData.prepare(evalSize=0.0,testSize=0.3,limitSize=None)

    for name,config in dnnConfigs.items():
        dnn = Dnn(mlData,'testPlots/mlPlots/asimovSigLoss/inclusive/'+varSetName+'/'+name)

        #First set up a model that trains on the sig loss
        dnn.setup(hiddenLayers=config['hiddenLayers'],dropOut=config['dropOut'],
                  l2Regularization=config['l2Regularization'], optimizer=config['optimizer'],
                  activation=config['activation'],
                loss=significanceLoss(expectedSignal,expectedBkgd),
                extraMetrics=[
                    asimovSignificanceLoss(expectedSignal,expectedBkgd,systematic),asimovSignificanceFull(expectedSignal,expectedBkgd,systematic),
                    significanceFull(expectedSignal,expectedBkgd),truePositive,falsePositive
                ])

        dnn.fit(epochs=5,batch_size=config['batch_size'])
        dnn.diagnostics(batchSize=config['batch_size'],subDir='pretraining')
        dnn.makeHepPlots(expectedSignal,expectedBkgd,[systematic],makeHistograms=False,subDir='pretraining')

        #Now recompile the model with a different loss and train further
        dnn.recompileModel(asimovSignificanceLoss(expectedSignal,expectedBkgd,systematic))
        dnn.fit(epochs=config['epochs'],batch_size=config['batch_size'])
        dnn.diagnostics(batchSize=config['batch_size'])
        dnn.makeHepPlots(expectedSignal,expectedBkgd,[systematic],makeHistograms=False)

        trainedModels[varSetName+'_asimovSigLoss_'+name]=dnn